In [2]:
from requests import get, post
from time import sleep
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/Artificial_proteins.csv")[['ID']]
df2 = df.copy()
df2['AF_match'] = ''
df2['AF_match_score'] = 0
df2['PDB_match'] = ''
df2['PDB_match_score'] = 0
df2['ticket'] = ''

In [ ]:
timer = 0

for index in range(len(df)):
    
    if df2.at[index, 'ticket'] != '':
        continue
        
    if timer > 20:
        if timer%5 == 1:
            sleep(200)

        if timer%30 == 0:
            sleep(300)

        if timer%50 == 0:
            sleep(400)

        if timer%100 == 1:
            sleep(500)
        
    print(index)
    pdb = open('Structures/' + df.at[index, 'ID'] + '.pdb', 'r').read()
    
    # open request
    ticket = post('https://search.foldseek.com/api/ticket', {
            'q' : pdb,
            'database[]' : ["afdb50", "pdb100"],
            'mode' : 'tmalign',
        }).json()
    
    while ticket["status"] == 'RATELIMIT':
        sleep(50)
        print("Rate limit")
        ticket = post('https://search.foldseek.com/api/ticket', {
            'q' : pdb,
            'database[]' : ["afdb50", "pdb100"],
            'mode' : 'tmalign',
        }).json()
    df2.at[index, 'ticket'] = ticket['id']
    timer += 1
    sleep(3)

In [76]:
df2.to_csv("data/Artificial_proteins_Foldseek.csv", index=False)

In [ ]:
for index in range(len(df2)):
    
    if df2.at[index, 'ticket'] == '':
        continue
        
    if df2.at[index, 'AF_match'] != '':
        continue
        
    print(index)
    
    # wait for response
    repeat = True
    sleep_time = 5
    while repeat:
        
        try:
            status = get('https://search.foldseek.com/api/ticket/' + df2.at[index, 'ticket']).json()
        except:
            sleep(sleep_time)
            sleep_time += 1
            continue
        # print(status)
        if status['status'] == "ERROR":
            # handle error
            print("ERROR")
            break

        # wait a short time between poll requests
        sleep(1)
        repeat = status['status'] != "COMPLETE"
    
    if status['status'] == "ERROR":
        df2.at[index, 'AF_match'] = ''
        continue
    
    # get all hits
    result = get('https://search.foldseek.com/api/result/' + df2.at[index, 'ticket'] + '/0').json()
    if result and result['results']:
        if result['results'][1] and result['results'][1]['alignments']:
            df2.at[index, 'PDB_match'] = f"{result['results'][1]['alignments'][0]['target']}"
            df2.at[index, 'PDB_match_score'] = result['results'][1]['alignments'][0]['eval']
        
        if result['results'][0] and result['results'][0]['alignments']:
            df2.at[index, 'AF_match'] = f"{result['results'][0]['alignments'][0]['target']}"
            df2.at[index, 'AF_match_score'] = result['results'][0]['alignments'][0]['eval']
    else:
        df2.at[index, 'AF_match'] = ""

In [78]:
df2.to_csv("data/Artificial_proteins_Foldseek.csv", index=False)